In [1]:
using ExcelReaders

In [2]:
workspace = pwd()
f = openxl(string(workspace,"/roadtrip.xls"))

ExcelFile <roadtrip.xls>

In [3]:
data_cities = readxl(f, "Donnees!A9:A17")

9×1 Array{Any,2}:
 "Montréal "   
 "Québec"      
 "TR"          
 " Sherbrooke "
 "Hull"        
 "Chicoutimi"  
 " Rimouski"   
 "Rouyn"       
 "Moncton"     

In [4]:
c = map(strip, data_cities)

9×1 Array{SubString{String},2}:
 "Montréal"  
 "Québec"    
 "TR"        
 "Sherbrooke"
 "Hull"      
 "Chicoutimi"
 "Rimouski"  
 "Rouyn"     
 "Moncton"   

In [5]:
data_distances = readxl(f, "Donnees!C9:K17")

9×9 Array{Any,2}:
   0.0  256.0  137.0  152.0   193.0  461.0   542.0   627.0   889.0
 256.0    0.0  125.0  209.0   422.0  206.0   313.0   848.0   719.0
 137.0  125.0    0.0  143.0   297.0  326.0   424.0   723.0   829.0
 152.0  209.0  143.0    0.0   337.0  414.0   484.0   776.0   737.0
 193.0  422.0  297.0  337.0     0.0  623.0   720.0   517.0  1075.0
 461.0  206.0  326.0  414.0   623.0    0.0   249.0   840.0   700.0
 542.0  313.0  424.0  484.0   720.0  249.0     0.0  1089.0   511.0
 627.0  848.0  723.0  776.0   517.0  840.0  1089.0     0.0  1552.0
 889.0  719.0  829.0  737.0  1075.0  700.0   511.0  1552.0     0.0

In [6]:
nb_row, nb_col = size(data)
distances = zeros(Int, nb_row, nb_col)
for i = 1:nb_row
    for j = 1:nb_col
        distances[i,j] = Int(data_distances[i,j])
    end
end
distances

UndefVarError: UndefVarError: data not defined

In [7]:
using JLD

In [8]:
save(string(workspace,"/distances.jld"),"dist",distances)
save(string(workspace,"/cities.jld"),"cities",c)

UndefVarError: UndefVarError: distances not defined

In [9]:
d = load(string(workspace,"/distances.jld"),"dist")
c = load(string(workspace,"/cities.jld"),"cities")

9×1 Array{SubString{String},2}:
 "Montréal"  
 "Québec"    
 "TR"        
 "Sherbrooke"
 "Hull"      
 "Chicoutimi"
 "Rimouski"  
 "Rouyn"     
 "Moncton"   

In [10]:
using JuMP

In [11]:
N = 9
m = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

In [12]:
@variable(m, x[1:N,1:N], Bin)

9×9 Array{Variable,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]  x[1,8]  x[1,9]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]  x[2,8]  x[2,9]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]  x[3,6]  x[3,7]  x[3,8]  x[3,9]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]  x[4,6]  x[4,7]  x[4,8]  x[4,9]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]  x[5,6]  x[5,7]  x[5,8]  x[5,9]
 x[6,1]  x[6,2]  x[6,3]  x[6,4]  x[6,5]  x[6,6]  x[6,7]  x[6,8]  x[6,9]
 x[7,1]  x[7,2]  x[7,3]  x[7,4]  x[7,5]  x[7,6]  x[7,7]  x[7,8]  x[7,9]
 x[8,1]  x[8,2]  x[8,3]  x[8,4]  x[8,5]  x[8,6]  x[8,7]  x[8,8]  x[8,9]
 x[9,1]  x[9,2]  x[9,3]  x[9,4]  x[9,5]  x[9,6]  x[9,7]  x[9,8]  x[9,9]

In [13]:
for k=1:N
    @constraint(m,x[k,k] == 0)
end

In [14]:
for i = 1:N
    for j = 1:N
        @constraint(m, sum(x[i,j] for i = 1:N) == 1)
        @constraint(m, sum(x[i,j] for j = 1:N) == 1)
    end
end

In [15]:
function subsets(x::Vector{T}) where T
    res = Vector{T}[[]]
    for elem in x, j in eachindex(res)
        push!(res, [res[j] ; elem])
    end
    return res
end

subsets (generic function with 1 method)

In [16]:
soustour = subsets([1:N;])

512-element Array{Array{Int64,1},1}:
 []                         
 [1]                        
 [2]                        
 [1, 2]                     
 [3]                        
 [1, 3]                     
 [2, 3]                     
 [1, 2, 3]                  
 [4]                        
 [1, 4]                     
 [2, 4]                     
 [1, 2, 4]                  
 [3, 4]                     
 ⋮                          
 [3, 5, 6, 7, 8, 9]         
 [1, 3, 5, 6, 7, 8, 9]      
 [2, 3, 5, 6, 7, 8, 9]      
 [1, 2, 3, 5, 6, 7, 8, 9]   
 [4, 5, 6, 7, 8, 9]         
 [1, 4, 5, 6, 7, 8, 9]      
 [2, 4, 5, 6, 7, 8, 9]      
 [1, 2, 4, 5, 6, 7, 8, 9]   
 [3, 4, 5, 6, 7, 8, 9]      
 [1, 3, 4, 5, 6, 7, 8, 9]   
 [2, 3, 4, 5, 6, 7, 8, 9]   
 [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
nb_tour = 0
for t in soustour
    T = length(t)
    if 2 ≤ T ≤ N-1
        nb_tour += 2
        @constraint(m, sum(x[t[k],t[k+1]] for k=1:T-1) + x[t[T],t[1]] ≤ T-1)
        @constraint(m, sum(x[t[k+1],t[k]] for k=1:T-1) + x[t[1],t[T]] ≤ T-1)
    end
end
nb_tour

1002

In [18]:
sum(2^i for i = 2:8)

508

In [19]:
@objective(m, Min, sum(d[i,j] * x[i,j] for i=1:N, j=1:N))

256 x[1,2] + 137 x[1,3] + 152 x[1,4] + 193 x[1,5] + 461 x[1,6] + 542 x[1,7] + 627 x[1,8] + 889 x[1,9] + 256 x[2,1] + 125 x[2,3] + 209 x[2,4] + 422 x[2,5] + 206 x[2,6] + 313 x[2,7] + 848 x[2,8] + 719 x[2,9] + 137 x[3,1] + 125 x[3,2] + 143 x[3,4] + 297 x[3,5] + 326 x[3,6] + 424 x[3,7] + 723 x[3,8] + 829 x[3,9] + 152 x[4,1] + 209 x[4,2] + 143 x[4,3] + 337 x[4,5] + 414 x[4,6] + 484 x[4,7] + 776 x[4,8] + 737 x[4,9] + 193 x[5,1] + 422 x[5,2] + 297 x[5,3] + 337 x[5,4] + 623 x[5,6] + 720 x[5,7] + 517 x[5,8] + 1075 x[5,9] + 461 x[6,1] + 206 x[6,2] + 326 x[6,3] + 414 x[6,4] + 623 x[6,5] + 249 x[6,7] + 840 x[6,8] + 700 x[6,9] + 542 x[7,1] + 313 x[7,2] + 424 x[7,3] + 484 x[7,4] + 720 x[7,5] + 249 x[7,6] + 1089 x[7,8] + 511 x[7,9] + 627 x[8,1] + 848 x[8,2] + 723 x[8,3] + 776 x[8,4] + 517 x[8,5] + 840 x[8,6] + 1089 x[8,7] + 1552 x[8,9] + 889 x[9,1] + 719 x[9,2] + 829 x[9,3] + 737 x[9,4] + 1075 x[9,5] + 700 x[9,6] + 511 x[9,7] + 1552 x[9,8]

In [20]:
m

Minimization problem with:
 * 1173 linear constraints
 * 81 variables: 81 binary
Solver is default solver

In [21]:
using Cbc

In [22]:
setsolver(m, CbcSolver())

In [23]:
status = solve(m)

:Optimal

In [24]:
sol = getvalue(x)

9×9 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

In [25]:
cities_visited = 1
city = 1
println(c[city])
while cities_visited ≠ N
    city = findfirst(sol[city,:] .== 1.0) 
    cities_visited += 1
    println(c[city])
end

Montréal
Hull
Rouyn
TR
Québec
Chicoutimi
Rimouski
Moncton
Sherbrooke
